# Библиотеки, используемые в системе для оценки недвижимости

In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import math
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina"
import warnings
warnings.filterwarnings('ignore')

# Функция расчета средней абсолютной процентной ошибки

In [ ]:
#Вычисляет среднюю абсолютную процентную ошибку
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np. array (y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Функция медианной абсолютной процентной ошибки

In [ ]:
#Вычисляет медианную абсолютную процентную ошибку
def median_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.median(np.abs((y_true - y_pred) / y_true)) * 100

# Функция вывода на экран рассчитанных метрик

In [ ]:
#Печатает рассчитанные значения коэффициента детерминации,
#средней и медианной абсолютных ошибок
def print_metrics(prediction, val_y):
    val_mae = mean_absolute_error(val_y, prediction)
    median_AE = median_absolute_error(val_y, prediction)
    r2 = r2_score(val_y, prediction)
print('')
print('R\u00b2: {:.2}'.format(r2))
print('')
print('Средняя абсолютная ошибка: {:.3} %'.format(mean_absolute_percentage_error(val_y, prediction)))
print ('Медианная абсолютная ошибка: {:.3} %'.format(median_absolute_percentage_error (val_y, prediction)))

# Отображение загруженных данных в виде таблицы

In [ ]:
file_path = path1
df = pd.read_csv(file_path)
#Выводим 5 первых строк датафрейма
df.head(5)

# Отображение загруженных данных в виде таблицы

In [ ]:
#Создаем новый столбец Стоимость 1 кв.м путем построчного деления стоимостей
#квартир на их общие площади
df['priceMetr'] = df['price']/df['totalArea']
#Задаем широту и долготу центра города и рассчитываем для каждой квартиры
#расстояние от центра и азимут
city_center_coordinates = [55.7522, 37.6156]
df['distance'] = list (map(lambda x, y: geodesic(city_center_coordinates,
[x, y]).meters, df['latitude'], df['longitude']))
df['azimuth'] = list(map(lambda x, y: get_azimuth(x, y), df['latitude'],
df['longitude']))

#Выбираем из датафрейма только те квартиры, которые расположены не дальше 40 км
#от центра города с панельными стенами
df = df.loc[(df['distance'] < 40000)]

#Округляем значения стоблцов Стоимости метра, расстояния и азимута
df['priceMetr'] = df['priceMetr'].round(0)
df['distance'] = df['distance'].round(0)
df['azimuth'] = df['azimuth'].round(0)

# Вывод информации о загруженных данных

In [ ]:
#Выводим сводную информацию о датафрейме и его столбцах (признаках)
df.info()

# Отсеивание данных с аномальным значениями

In [ ]:
#Вычисляем строки со значениями- выбросами
first_quartile = df.quantile(q=0.25)
third_quartile = df.quantile(q=0.75)
IQR = third_quartile - first_quartile
outliers = df[(df › (third_quartile + 1.5 * IQR)) | (df < (first_quartile - 1.5 * IQR))].count(axis=1)
outliers.sort_values(axis=0, ascending=False, inplace=True)

#Удаляем из датафрейма 3000 строк, подходящих под критерии выбросов
outliers = outliers.head(3000)
df.drop(outliers.index, inplace=True)

# Кодирование категориальных признаков в числа

In [ ]:
#Вычисляем столбцы с категорийными признаками, затем заменяем их на числа
categorical_columns = df.columns[df.dtypes == 'object']
labelencoder = LabelEncoder()
for column in categorical_columns:
    df[column] = labelencoder.fit_transform(df[column])
    print(dict(enumerate(labelencoder.classes_)))

# Вывод информации о загруженных данных после кодирования категориальных признаков

In [ ]:
#Выводим сводную информацию о датафрейме и его столбцах (признаках), чтобы
#убедиться, что теперь они все содержат цифровые значения
df.info()

# Формирование матрицы X с атрибутами квартир и вектора y c соответствующими ценами за квадратный метр

In [ ]:
#Назначаем целевой переменной цену 1 кв. метра, а можно и цену всей квартиры,
#тогда будет y = df['price']
y = df['priceMetr']

#Создаем список признаков, на основании которых будем строить модели
features = [
    'wallsMaterial',
    'floorNumber',
    'floorsTotal',
    'totalArea',
    'kitchenArea',
    'distance',
    'azimuth'
    ]

#Создаем датафрейм, состоящий из признаков, выбранных ранее
X = df[features]

# Формирование тренировочной и тестовой выборок данных

In [ ]:
#Проводим случайное разбиение данных на выборки для обучения (train)
#и валидации (val), по умолчанию в пропорции 0.75/0.25
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

# Обучение регрессионной модели Random forest и расчет ошибки прогнозирования

In [ ]:
#Создаем регрессионную модель случайного леса
rf_model = RandomForestRegressor(n_estimators=2000, #2000
                                 n_jobs=-1,
                                 bootstrap=False,
                                 criterion='friedman_mse',
                                 max_features=3,
                                 random_state=1,
                                 max_depth=55,
                                 min_samples_split=5
                                 )

#Проводим подгонку модели на обучающей выборке
rf_model.fit(train_X, train_y)

#Вычисляем предсказанные значения цен на основе валидационной выборки
rf_prediction = rf_model.predict(val_X).round(0)

#Вычисляем и печатаем величины ошибок при сравнении известных цен квартир из
#валидационной выборки с предсказанными моделью
print_metrics(rf_prediction, val_y)

# Обучение регрессионной модели XGBoost и расчет ошибки прогнозирования

In [ ]:
#Создаем регрессионную модель XGBoost
xgb_model = xgb.XGBRegressor(objective ='reg:gamma',
                             learning_rate = 0.01,
                             max_depth = 45,
                             n_estimators = 2000,
                             nthread = -1,
                             eval_metric = 'gamma-nloglik',
)

#Проводим подгонку модели на обучающей выборке
xgb_model.fit(train_X, train_y)

#Вычисляем предсказанные значения цен на основе валидационной выборки
xgb_prediction = xgb_model. predict(val_X).round(0)

#Вычисляем и печатаем величины ошибок при сравнении известных цен квартир из
#валидационной выборки с предсказанными моделью
print_metrics(xgb_prediction, val_y)

# Использование гибридной модели

In [ ]:
#Усредняем предсказания обоих моделей
prediction = rf_prediction * 0.5 + xgb_prediction * 0.5

#Вычисляем и печатаем величины ошибок для усредненного предсказания
print_metrics(prediction, val_y)

# Расчет значимости признаков и построение диаграммы

In [ ]:
#Рассчитываем важность признаков в модели Random forest
importances = rf_model.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf_model.estimators_],
             axis=0)
indices = np.argsort(importances) [::-1]

#Печатаем рейтинг признаков
print("Рейтинг важности признаков:")
for f in range(X.shape[1]):
    print("%d. %s (%f)" % (f + 1, features[indices[f]], importances[indices[f]]))

#Строим столбчатую диаграмму важности признаков
plt.figure()
plt.title ("Важность признаков")
plt.bar(range(X.shape[1]), importances[indices], color="g", yerr=std[indices],
        align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt. show()

# Задание параметров квартиры и расчет прогнозной цены

In [ ]:
flat = pd.DataFrame({
    'wallsMaterial':[6],
    'floorNumber':[4],
    'floorsTotal':[17],
    'totalArea' :[51.2],
    'kitchenArea':[9.7],
    'latitude':[55.858817],
    'longitude':[37.638755]
    })

#Рассчитываем недостающие параметры квартиры - расстояние от центра города и азимут
flat['distance'] = list(map(lambda x, y: geodesic(city_center_coordinates,
                                                  [x, y]).meters, flat['latitude'],
                                                  flat['longitude']))
flat['azimuth'] = list(map(lambda x, y: get_azimuth(x, y), flat['latitude'],
                           flat['longitude']))
flat['distance'] = flat['distance'].round(0)
flat['azimuth'] = flat['azimuth'].round(0)

#Удаляем ненужные столбцы с широтой и долготой
flat = flat.drop('latitude', axis=1)
flat = flat.drop('longitude', axis=1)

#Вычисляем предсказанное значение стоимости по двум моделям
rf_prediction_flat = rf_model.predict(flat).round(0)
xgb_prediction_flat = xgb_model.predict(flat).round(0)

#Усредняем полученные знаечения и умножаем на общую площадь квартиры
price = (rf_prediction_flat * 0.5 + xgb_prediction_flat * 0.5)*flat['totalArea'][0]

#Печатаем предсказанное значение цены предложения
print(f'Предсказанная моделью цена предложения: {int(price[0].round(-3))} рублей')